The experiment in this notebook is to replicate demos in https://colab.research.google.com/drive/1-HDJUcPMKEF-E7Hapih0OmA1xTW2hdAv#scrollTo=p9en0SOhRaGz

### Toxicity

In [1]:
from datasets import load_dataset

toxicity_prompts=load_dataset("allenai/real-toxicity-prompts", split="train")

In [2]:
import random

def get_random_prompts(dataset, num_examples=100):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    return(dataset[picks])

In [5]:
test_sample= get_random_prompts(toxicity_prompts,100)
test_prompts = [p['text'] for p in test_sample['prompt']]

In [4]:
from transformers import pipeline, AutoTokenizer

text_generation = pipeline("text-generation", model="gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [6]:
model_continuations=[]
for prompt in test_prompts:
  generation = text_generation(prompt, max_length=50, do_sample=False, pad_token_id=231219)
  continuation = generation[0]['generated_text'].replace(prompt,'')
  model_continuations.append(continuation)

Generate 100 continuation cost 11 minutes and 54.7 seconds

In [7]:
import evaluate 
toxicity = evaluate.load("toxicity")

Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint


config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

c:\Users\15783\source\repos\Ethical-Framework\Evaluate\.venv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\15783\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [8]:
toxicity_ratio = toxicity.compute(predictions=model_continuations, aggregation="ratio")
print(toxicity_ratio)

{'toxicity_ratio': 0.02}


In [9]:
all_toxicity=toxicity.compute(predictions=model_continuations)

In [13]:
import math
avg_toxicity=sum(all_toxicity["toxicity"])/len(all_toxicity["toxicity"])
display(avg_toxicity)

0.04106289582327008